In [ ]:
# 2022测试集输出格式转换
import pandas as pd
test = pd.read_excel('/home/liyunliang/CGED_Task/output/Grammar_detect_task_predict_22/Result/test_prediction-2022-09-18|22:22:34-test-0-Test_mode.xlsx')
add_test = open('/home/liyunliang/CGED_Task/cged.pred.txt.bak').readlines()
add_test_dict = {}
for add in add_test:
    add = add.strip().split(',\t')
    assert len(add) in [2,4,5]
    if len(add) in [2]:
        # correct
        add_test_dict[add[0]] = {}
    elif len(add) in [4]:
        # R/W
        if add[0] not in add_test_dict:
            add_test_dict[add[0]] = {}
        add_test_dict[add[0]][(add[1], add[2], add[3])] = ''
    else:
        # S/M
        if add[0] not in add_test_dict:
            add_test_dict[add[0]] = {}
        add_test_dict[add[0]][(add[1], add[2], add[3])] = add[4]
# add_test_dict
cnt = 0
res = []
for index in test.index:
    id_ = str(test.iloc[index]['id'])
    position = eval(test.iloc[index]['position_pred'])
    # if not position or add_test_dict[id_]=={}:
    if not position:
        res.append(',\t'.join([id_, 'correct']))
    # print(type(eval(position)[0]))
    else:
        position = sorted(position, key=lambda x: x['position'][0])
        # print(position)
        for pos in position:
            if pos['error_type'] in ['R', 'W']:
                res.append(',\t'.join([id_, str(pos['position'][0]+1), str(pos['position'][1]), pos['error_type']]))
            elif add_test_dict[id_]!= {} and (str(pos['position'][0]+1), str(pos['position'][1]), pos['error_type']) in add_test_dict[id_] \
                and add_test_dict[id_][(str(pos['position'][0]+1), str(pos['position'][1]), pos['error_type'])] != '':
                cnt += 1
                res.append(',\t'.join([id_, str(pos['position'][0]+1), str(pos['position'][1]), pos['error_type'], add_test_dict[id_][(str(pos['position'][0]+1), str(pos['position'][1]), pos['error_type'])]]))
            else:
                res.append(',\t'.join([id_, str(pos['position'][0]+1), str(pos['position'][1]), pos['error_type']]))
    
print(cnt)
# 确认预测结果的数目没有问题
print(len(set([item.split(',\t')[0] for item in res])))
open('/home/liyunliang/CGED_Task/cged_pred_check.txt', 'w').write('\n'.join(res))
